In [14]:
# coding:u8
import json
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
import gc
from gensim.models import Word2Vec
from sklearn.model_selection import KFold

In [15]:
# train_data = []
# train_data_dict = {}
# with open('data/raw_data/train.json','r') as f:
#     for line in f:
#         raw = json.loads(line)
#         train_data.append(raw)
#         train_data_dict[raw['text_id']] = raw
# # 读取知识库
# kb_data = []
# kb = {}
# with open('data/raw_data/kb_data','r') as f:
#     for line in f:
#         item = json.loads(line)
#         kb[item['subject_id']] = item
#         kb_data.append(item)


In [16]:
train_data = []
train_data_dict = {}
with open('data/raw_data/train.json', 'r') as f:
    for line in f:
        raw = eval(str(json.loads(line)).lower())
        train_data.append(raw)
        train_data_dict[raw['text_id']] = raw
kb = {}
with open('data/raw_data/kb_data', 'r') as f:
    for line in f:
        item = eval(str(json.loads(line)).lower())
        kb[item['subject_id']] = item
# 补充别名实体进入kb
for s in train_data:
    for m in s['mention_data']:
        if m['kb_id'] == 'nil': continue
        if m['mention'] != kb[m['kb_id']]['subject'] and m['mention'] not in kb[m['kb_id']]['alias']:
            kb[m['kb_id']]['alias'] += [m['mention']]
            
name_id = {}
for kb_id in kb:
    for item in kb[kb_id]['alias']:
        if item not in name_id:
            name_id[item] = [kb_id]
        else:
            name_id[item].append(kb_id)
    if kb[kb_id]['subject'] not in name_id:
        name_id[kb[kb_id]['subject']] = [kb_id]
    else:
        name_id[kb[kb_id]['subject']].append(kb_id)
for id in name_id:
    name_id[id] = list(set(name_id[id]))

In [ ]:
kb['341467']

In [ ]:
# name_id

In [6]:
"""
TODO:
预测type,
引入tfidf
"""

'\nTODO:\n预测type,\n引入tfidf\n'

In [6]:
train_data = train_data

In [11]:
# K = 0
# for s in tqdm(train_data):
#     mention_ner = s['mention_data']
#     for m in mention_ner:
#         if m['mention'] not in name_id:
#             continue
#         candidate_ids = name_id[m['mention']]
#         label = 0
#         for m_candidate_id in candidate_ids:
           
#             if m_candidate_id==m['kb_id']:
#                 label = 1
#                 K+=1
#         if label==0:
#             print(m)

In [4]:
# max(data['m_id'])

In [18]:
kb_column = []
train_column = []
text_id = []
label_colum = []
num_attrs = []
num_abstract_words = []
num_alias = []
m_id = 0
m_id_list = []
num_candidate = []
equal_subject = []
shuminghao = [] 
entity_common = []
len_mention = []
mention_start = []
is_eng = []
for s in tqdm(train_data):
    mention_ner = s['mention_data']
    m_list = []
    for m in mention_ner:
        m_list.append(m['mention'])
    for m in mention_ner:
        if m['mention'] not in name_id: # 不能注销，因为有些实体没有or m['kb_id']=='nil':
            continue
        candidate_ids = name_id[m['mention']]
        for m_candidate_id in candidate_ids:
            # 统计配对特征
            candidate_numattrs = 0
            candidate_abstract_numwords = 0
            candidate_detail = kb[m_candidate_id]
            candi_text = ''
            for predicate in candidate_detail['data']:
                candidate_numattrs += 1
                candi_text += predicate['object']
                if predicate['predicate'] == '摘要':
                    candidate_abstract = predicate['object']
                    candidate_abstract_numwords = len(candidate_abstract)
#                 if predicate['predicate'] == '标签':
#                     candidate_label += predicate['object']
            if m_candidate_id==m['kb_id']:
                label = 1
            else:
                label = 0
            train_column.append(m['mention'])
            kb_column.append(m_candidate_id)
            text_id.append(s['text_id'])
            label_colum.append(label)
            num_attrs.append(candidate_numattrs)
            num_abstract_words.append(candidate_abstract_numwords)
            num_alias.append(len(set(candidate_detail['alias']+[candidate_detail['subject']])))
            m_id_list.append(m_id)
            num_candidate.append(len(candidate_ids))
            equal_subject.append(candidate_detail['subject']==m['mention'])
            shuminghao.append('《'+candidate_detail['subject']+'》' in candidate_abstract)
#             if langid.classify(m['mention'])[0]=='en':
#                 len_mention.append(len(m['mention'].split()))
#                 is_eng.append(1)
#             else:
#                 len_mention.append(len(m['mention']))
#                 is_eng.append(0)
            len_mention.append(len(m['mention']))
            mention_start.append(m['offset'])
            
            i = 0
            for item in m_list:
                if item in candi_text: i+=1
            entity_common.append(i)
        m_id += 1
        
data = pd.DataFrame()
data['text_id'] = text_id
data['kb_id'] = kb_column
data['train_mention'] = train_column
data['label'] = label_colum
data['num_attrs'] = num_attrs
data['num_abstract_words'] = num_abstract_words
data['num_alias'] = num_alias
data['m_id'] = m_id_list
data['num_candidates'] = num_candidate 
data['mention_equal_subject'] = equal_subject
data['shuminghao'] = shuminghao
data['entity_common']  = entity_common
data['len_mention'] = len_mention
data['mention_start'] = list(map(int,mention_start))

100%|██████████| 90000/90000 [00:19<00:00, 4589.95it/s]


In [8]:
mention_vector = np.load('entity_embedding/gensim_vector.npy')
entity_embedding = Word2Vec.load('embedding/w2v.model')
from sklearn.metrics import mean_squared_error as mse

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
def cal_mse(kb_id,m_id):
    return mse(entity_embedding.wv.word_vec(kb_id),mention_vector[int(m_id)])

In [ ]:
data['mse_distance'] = data.apply(lambda x:cal_mse(x.kb_id,x.m_id),axis=1)

In [54]:
data.head(20)

,text_id,kb_id,train_mention,label,num_attrs,num_abstract_words,num_alias,m_id,num_candidates,mention_equal_subject,...,product,scientificorganization,symbol,theorem,thing,tool,tvplay,tvshow,vocabulary,zodiacsign
0,1,311223,南京南站,1,17,447,3,0,2,True,...,0,0,0,0,0,0,0,0,0,0
1,1,130287,南京南站,0,9,338,2,0,2,True,...,0,0,0,0,0,0,0,0,0,0
2,1,213561,高铁,0,1,0,2,1,2,False,...,0,0,0,0,1,0,0,0,0,0
3,1,341096,高铁,1,10,239,1,1,2,True,...,0,0,0,0,1,0,0,0,0,0
4,1,311223,南京南站,1,17,447,3,2,2,True,...,0,0,0,0,0,0,0,0,0,0
5,1,130287,南京南站,0,9,338,2,2,2,True,...,0,0,0,0,0,0,0,0,0,0
6,1,311223,南京南站,1,17,447,3,3,2,True,...,0,0,0,0,0,0,0,0,0,0
7,1,130287,南京南站,0,9,338,2,3,2,True,...,0,0,0,0,0,0,0,0,0,0
8,2,278410,比特币,1,12,610,4,4,4,True,...,0,0,0,0,0,0,0,0,0,0
9,2,91051,比特币,0,9,49,3,4,4,False,...,0,0,0,0,0,0,0,0,0,0


In [26]:
m_id

257078

In [43]:
data.head()

,text_id,kb_id,train_mention,label,num_attrs,num_abstract_words,num_alias,m_id,num_candidates,mention_equal_subject,...,scientificorganization,symbol,theorem,thing,tool,tvplay,tvshow,vocabulary,zodiacsign,mse_distance
0,1,311223,南京南站,1,17,447,3,0,2,True,...,0,0,0,0,0,0,0,0,0,0.003157
1,1,130287,南京南站,0,9,338,2,0,2,True,...,0,0,0,0,0,0,0,0,0,0.002699
2,1,213561,高铁,0,1,0,2,1,2,False,...,0,0,0,1,0,0,0,0,0,0.000728
3,1,341096,高铁,1,10,239,1,1,2,True,...,0,0,0,1,0,0,0,0,0,0.002646
4,1,311223,南京南站,1,17,447,3,2,2,True,...,0,0,0,0,0,0,0,0,0,0.003268


In [28]:
# # 距离特征，和　mention的距离
# def cal_dis()
# dis_candi_mention = []
# data['dis_candi_mention'] = data.apply(lambda x:mse(entity_embedding.wv.word_vecx['kb_id']))

#             candi_vector = entity_embedding.wv.word_vec(kb[m_candidate_id]['subject_id'])
#             m_vector = 
#             dis_candi_mention# 

In [29]:
#todo rank 特征

In [19]:
data['type'] = data.apply(lambda x:kb[x['kb_id']]['type'][0],axis=1)
data = pd.concat([data,pd.get_dummies(data['type'])],axis=1)

In [25]:
data.head()

,text_id,kb_id,train_mention,label,num_attrs,num_abstract_words,num_alias,m_id,num_candidates,mention_equal_subject,...,product,scientificorganization,symbol,theorem,thing,tool,tvplay,tvshow,vocabulary,zodiacsign
0,1,130287,南京南站,0,9,338,2,0,2,True,...,0,0,0,0,0,0,0,0,0,0
1,1,311223,南京南站,1,17,447,3,0,2,True,...,0,0,0,0,0,0,0,0,0,0
2,1,213561,高铁,0,1,0,2,1,2,False,...,0,0,0,0,1,0,0,0,0,0
3,1,341096,高铁,1,10,239,1,1,2,True,...,0,0,0,0,1,0,0,0,0,0
4,1,130287,南京南站,0,9,338,2,2,2,True,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# 流行度特征
# 总流行度
data_ctr = []
kfold = KFold(n_splits=9,shuffle=False,random_state=2019)
for train_index,test_index in kfold.split(data):
    print(test_index)
    stat_train_data,stat_test_data = data.loc[train_index,:],data.loc[test_index,:]
    #print(test_data.head())
    train_group = stat_train_data.groupby('kb_id',as_index=False)['label'].agg({'label_mean':'mean','label_count':'count'})
    stat_test_data = stat_test_data.merge(train_group,on='kb_id',how='left')
    print(stat_test_data[stat_test_data.label_mean.isnull()].shape)
    train_group = stat_train_data.groupby(['kb_id','train_mention'])['label'].agg({'m_label_mean':'mean','m_label_count':'count'})
    stat_test_data = stat_test_data.merge(train_group,on=['kb_id','train_mention'],how='left')
    data_ctr.append(stat_test_data)
data_ctr = pd.concat(data_ctr,axis=0)


[     0      1      2 ... 203883 203884 203885]
(20029, 65)


/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if sys.path[0] == '':


[203886 203887 203888 ... 407769 407770 407771]
(19987, 65)
[407772 407773 407774 ... 611655 611656 611657]
(19324, 65)
[611658 611659 611660 ... 815540 815541 815542]
(20262, 65)
[ 815543  815544  815545 ... 1019425 1019426 1019427]
(19479, 65)
[1019428 1019429 1019430 ... 1223310 1223311 1223312]
(19668, 65)
[1223313 1223314 1223315 ... 1427195 1427196 1427197]
(19561, 65)
[1427198 1427199 1427200 ... 1631080 1631081 1631082]
(19584, 65)
[1631083 1631084 1631085 ... 1834965 1834966 1834967]
(19649, 65)


In [21]:
to_rank = ['num_attrs', 'num_abstract_words', 'num_alias', 'label_mean', 'label_count', 'm_label_mean', 
             'm_label_count','num_candidates','mention_equal_subject','shuminghao','entity_common']
for f in to_rank:
    data_group = data_ctr.groupby('m_id')[f].rank(ascending=False)
    data_ctr['rank_'+f] = data_group

In [22]:
data_ctr.columns

Index(['text_id', 'kb_id', 'train_mention', 'label', 'num_attrs',
       'num_abstract_words', 'num_alias', 'm_id', 'num_candidates',
       'mention_equal_subject', 'shuminghao', 'entity_common', 'len_mention',
       'mention_start', 'type', 'academicdiscipline', 'animal',
       'astronomicalobject', 'athlete', 'awardeventseries', 'brand',
       'building', 'collegeoruniversity', 'communicationmedium', 'country',
       'creativework', 'culturalheritage', 'currency', 'curriculum',
       'educationmajor', 'entertainmentperson', 'event', 'familyname',
       'fictionalhuman', 'fictionalthing', 'food', 'formula', 'game',
       'historicalperiod', 'historicalperson', 'human',
       'internationalorganization', 'language', 'material', 'medicalcondition',
       'medicaldepartmenttype', 'movie', 'nation', 'organism', 'organization',
       'person', 'place', 'plant', 'product', 'scientificorganization',
       'symbol', 'theorem', 'thing', 'tool', 'tvplay', 'tvshow', 'vocabulary',
   

In [23]:
data_ctr.reset_index(drop=True,inplace=True)
data_ctr.to_pickle('features/1_stat_feature.pkl')